In [1]:
import polars as pl
df = pl.read_csv("training_data/train.csv", encoding="utf-8")
df.head()

movieid,reviewerName,isFrequentReviewer,reviewText,sentiment
str,str,bool,str,str
"""marvelous_pira…","""Benjamin Henry…",false,"""Henry Selick’s…","""POSITIVE"""
"""tony_montana_f…","""Felicia Lopez""",false,"""With a cast th…","""NEGATIVE"""
"""darth_vader_ka…","""Mr. Charles Bu…",true,"""Creed II does …","""POSITIVE"""
"""lara_croft_gli…","""Ryan Barrett""",false,"""I know what yo…","""POSITIVE"""
"""jason_bourne_s…","""Alexander Glov…",false,"""Director Ferna…","""POSITIVE"""


In [2]:
from quick_sentiments import pre_process
from quick_sentiments.vect.wv import vectorize

NLTK data 'punkt' already present.
NLTK data 'stopwords' already present.


[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\meala\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\meala\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


NLTK data 'wordnet' downloaded.
NLTK data 'omw-1.4' downloaded.


### Word Embedding, Word2Vec
We are using Word2Vec for word embedding, which is a popular technique to convert words into numerical vectors. This allows us to capture semantic relationships between words. Word2Vec assigns 300 dimensions to each word, which means each word is represented by a vector of 300 numbers. The sentence is then represented by the average of these word vectors.

While there are other dimension of word2vec, we are using 300 dimensions here for simplicity and effectiveness. Further updates in this packages will allow you to choose different dimensions.

In [3]:
wv_demo =[pre_process(text) for text in df["reviewText"][:5]]
wv_demo

['henry selicks first movie since 2009s coraline fifth stopmotion masterpiece',
 'cast read like vogue oscar party guest list valentine day cantmiss cinema instead standard hollywood schmaltz',
 'creed ii give u anything another slightly superior rocky sequel win point nt expect knockout',
 'know thinking limitless bradley cooper cell multiply lucy tap brain new thrillsnew skill passing hour',
 'director fernando meirelles tell story urgency sharp visual composition washed cinematography ooze gangster life']

In [6]:
wv_df, vect = vectorize(wv_demo)

Using already loaded Word2Vec model.


Unlike Bag of Words (BOW) or Term Frequency-Inverse Document Frequency (TF-IDF), which treat words as independent entities, Word2Vec captures the context and meaning of words based on their usage in sentences. This makes it particularly useful for tasks like sentiment analysis, where understanding the nuances of language is crucial.The columns are always fixed at 300, and every columns has some values.

Words like (King, Queen, and Royalty), (Football, Scoccer, Real Madrid ) will have similar vectors because they are related in meaning. This is different from BOW or TF-IDF, where these words would be treated as completely separate entities without any relationship.

In [10]:
pl.DataFrame(wv_df)


column_0,column_1,column_2,column_3,column_4,column_5,column_6,column_7,column_8,column_9,column_10,column_11,column_12,column_13,column_14,column_15,column_16,column_17,column_18,column_19,column_20,column_21,column_22,column_23,column_24,column_25,column_26,column_27,column_28,column_29,column_30,column_31,column_32,column_33,column_34,column_35,column_36,…,column_263,column_264,column_265,column_266,column_267,column_268,column_269,column_270,column_271,column_272,column_273,column_274,column_275,column_276,column_277,column_278,column_279,column_280,column_281,column_282,column_283,column_284,column_285,column_286,column_287,column_288,column_289,column_290,column_291,column_292,column_293,column_294,column_295,column_296,column_297,column_298,column_299
f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,…,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32
0.121811,-0.063228,0.02774,0.147257,0.028666,-0.044922,-0.053426,-0.106771,0.161702,0.103353,-0.032267,-0.192586,0.001648,-0.00531,-0.038111,0.022776,0.100484,-0.00297,0.031464,-0.113098,-0.091064,0.036326,0.047241,-0.08311,0.019307,-0.006526,-0.02537,0.041056,0.070719,0.07988,-0.105143,0.015381,-0.077037,0.026245,0.062622,0.021006,-0.035492,…,-0.016432,0.039754,0.080495,-0.035461,-0.100342,-0.088338,0.048581,0.017581,0.043493,0.06722,0.0097,0.031453,-0.019002,-0.129557,-0.099935,-0.113851,-0.009511,0.013346,0.050293,-0.011739,0.118123,-0.037679,-0.036133,-0.01652,0.031631,-0.018026,0.061605,-0.186239,0.04835,-0.07312,0.075785,-0.037577,-0.118449,-0.205892,-0.079191,-0.041555,-0.018717
-0.046672,0.03339,-0.029568,0.18689,-0.002755,0.033523,0.04598,-0.068477,0.0368,0.074364,-0.05592,-0.124597,0.003141,0.041557,-0.089038,0.163417,0.143962,0.010482,-0.01522,-0.116463,0.042025,0.075116,-0.030497,-0.024801,0.030888,-0.044602,-0.004003,0.047009,0.06272,-0.046235,-0.072567,-0.031584,-0.041561,0.051648,0.013696,-0.040101,0.117185,…,-0.108919,0.065291,-0.001408,-0.060986,-0.024573,-0.087297,-0.082209,-0.024237,-0.013215,0.075195,-0.003981,0.107926,-0.032943,-0.001843,-0.064366,-0.094613,0.056836,-0.001318,-0.006489,-0.063053,0.021016,0.018457,0.066636,-0.053204,-0.019059,0.029329,0.082474,-0.019424,0.029386,-0.108862,0.006136,0.044279,-0.103255,-0.001807,-0.004173,-0.057788,0.110409
-0.001188,0.004293,0.054455,0.10249,-0.042594,0.01414,0.059619,-0.113798,0.071501,0.096033,-0.073017,-0.10592,-0.063818,0.070465,-0.07675,0.140513,0.050948,0.065393,0.033622,-0.069922,-0.05755,0.032113,0.11259,-0.077096,0.02085,-0.065792,-0.086735,0.013055,0.051172,0.042586,-0.055257,-0.000372,-0.037606,-0.038314,-0.060124,0.11096,-0.030509,…,0.058508,0.067041,0.108415,-0.004814,-0.102637,-0.069991,-0.054533,0.057369,0.055971,0.084781,-0.028939,0.013509,0.022534,-0.067741,-0.086218,-0.052922,0.083161,-0.002106,0.024951,0.013102,0.04292,0.0125,0.00423,0.002537,0.004026,-0.056613,-0.001624,-0.000343,0.073832,-0.089646,-0.010278,-0.018433,-0.050814,-0.043254,-0.137549,-0.01383,0.011507
-0.013966,0.055578,-0.022818,0.101641,-0.077794,0.045044,0.013499,-0.129216,0.024423,0.076974,-0.040196,-0.071023,-0.035823,-0.019283,-0.091612,0.086269,-0.042899,0.229213,0.007586,-0.059893,-0.028107,-0.005167,-0.035444,0.048453,-0.020874,0.005092,-0.078683,0.012669,0.045115,-0.024879,-0.05808,0.02291,-0.057059,0.029672,-0.074772,-0.054265,-0.022736,…,0.026635,-0.032532,0.075799,0.018258,-0.05161,-0.086042,-0.005598,-0.085528,0.071368,0.056502,0.006627,0.064139,-0.022252,-0.059239,-0.046092,-0.030562,0.096366,0.011032,0.071764,0.062884,-0.026697,0.003501,-0.005977,-0.045013,0.026124,0.146624,0.146803,0.062622,0.084211,-0.075858,-0.063703,-0.138733,-0.12364,0.004508,-0.058173,-0.073382,0.052028
0.070343,-0.000009,0.028959,-0.007155,-0.039034,0.075545,0.081604,-0.082651,0.063308,0.026639,-0.008204,-0.129202,-0.036457,0.077956,-0.1407